# Load data

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
import csv

In [2]:
# filenames
HOME = '../load_data/'

INPUT_TRAIN = '{}input_train.csv'.format(HOME)
INPUT_TEST = '{}input_test.csv'.format(HOME)
OUTPUT_TRAIN = '{}output_train-1.csv'.format(HOME)
OUTPUT_TEST = '{}output_test-1.csv'.format(HOME)
OUTPUT_GENES = '{}output_genes-1.txt'.format(HOME)

MSE_OUTPUT = '37x37_pearson_mse.csv'

In [3]:
# load data into dataframes
train_input = pd.read_csv(INPUT_TRAIN, header=0, index_col=0)
train_output = pd.read_csv(OUTPUT_TRAIN, header=0, index_col=0)
test_input = pd.read_csv(INPUT_TEST, header=0, index_col=0)
test_output = pd.read_csv(OUTPUT_TEST, header=0, index_col=0)

## 37x37 input, TFs chosen based on Pearson correlation to output genes

In [31]:
# choose 37^2 most correlated transcription factors for each output gene
# reshape input dataframe into 37*37*1 arrays
input_df = pd.concat([train_input, test_input], axis=0)
output_df = pd.concat([train_output, test_output], axis=0)
cor_input_train = dict()
cor_input_test = dict()
for gene in test_output.columns:
    cor = input_df.corrwith(output_df[gene])
    tfs = cor.nlargest(37**2).index
    cor_input_train[gene] = train_input[tfs].to_numpy().reshape(18542, 37, 37, 1)
    cor_input_test[gene] = test_input[tfs].to_numpy().reshape(4636, 37, 37, 1)

In [33]:
# train VGG16 model for each gene
metrics = dict()

for gene in genes:
    print(gene)
    # model
    model = VGG16(include_top=False, weights=None, input_shape=(37,37,1))
    x = Flatten()(model.output)
    x = Dense(units=4096, activation='relu')(x)
    x = Dense(units=4096, activation='relu')(x)
    x = Dense(units=1)(x)
    model = Model(inputs=model.inputs, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

    # training
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    model.fit(x=cor_input_train[gene], y=train_output[gene], epochs=50, callbacks=[early_stop], validation_split=.2)

    # evaluation
    metrics[gene] = (model.evaluate(x=cor_input_test[gene], y=test_output[gene])[0])

App
Train on 14833 samples, validate on 3709 samples
Epoch 1/50
14833/14833 [==============================] - 28s 2ms/sample - loss: 3.4728e-06 - mse: 3.4728e-06 - val_loss: 1.9376e-07 - val_mse: 1.9376e-07
Epoch 2/50
14833/14833 [==============================] - 27s 2ms/sample - loss: 3.9451e-07 - mse: 3.9451e-07 - val_loss: 1.7575e-07 - val_mse: 1.7575e-07
Epoch 3/50
14833/14833 [==============================] - 27s 2ms/sample - loss: 3.4634e-07 - mse: 3.4634e-07 - val_loss: 3.4701e-07 - val_mse: 3.4701e-07
Epoch 4/50
14833/14833 [==============================] - 27s 2ms/sample - loss: 2.9596e-07 - mse: 2.9596e-07 - val_loss: 1.9159e-07 - val_mse: 1.9159e-07
Epoch 5/50
14833/14833 [==============================] - 27s 2ms/sample - loss: 3.0146e-07 - mse: 3.0146e-07 - val_loss: 2.2687e-07 - val_mse: 2.2687e-07
Epoch 6/50
14833/14833 [==============================] - 27s 2ms/sample - loss: 3.0158e-07 - mse: 3.0158e-07 - val_loss: 6.1904e-07 - val_mse: 6.1904e-07
Epoch 7/50
4636/4

KeyError: 'Map'

In [34]:
metrics

{'App': 2.8114254574811375e-07,
 'Apoe': 9.93106450449807e-06,
 'Gusb': 2.121164140815287e-08,
 'Lamp5': 5.076448983412173e-07,
 'Mbp': 7.320084201043031e-07,
 'Pvalb': 2.0035805047576416e-07,
 'Rorb': 1.9169013785054322e-07,
 'S100b': 1.6933947452792726e-08,
 'Slc30a3': 1.3342836044360167e-08,
 'Snca': 7.870812282032773e-07}

In [35]:
# write mse to csv
with open(MSE_OUTPUT, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(metrics.items())